In [2485]:
rutaRelativa = rutaRelativa.replace('\\SRC', '')
rutaRelativa

'/Users/camilo/Desktop/PROYECTOS'

In [2486]:
dbAzure = DBAzure(estadoProduccion)

CONEXION CON EL SERVIDOR


### Obtengo la información de los equipos por enviar

In [2487]:
equipos = dbAzure.getQuerySolpeds()
#solpeds
display(equipos)

/Users/camilo/Desktop/PROYECTOS/ROTACION/classes/dbAzure.py:65: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(self.query, self.connection)


,id,nombreMaquina,temperaturaMinima,temperaturaMaxima,nombreLocal,TIPO_DE_EQUIPO,ZONA,NUMERO_SERIE,ESTADO,OTRO,OBSERVACION,MODELO,Creado,DESCRIPCION,ESTADO_CORREO,ORDEN_COMPRA,Modificado,Observaciones,created_at,updated_at
0,338,8,-18,-12,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,NO OPERATIVO,0,prueba,,,,POR_ENVIAR,3778731,,,2023-03-22 11:23:40.000,2023-03-22 11:23:40.000
1,90,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,NO OPERATIVO,494,,None,02/02/2023 6:14,None,POR_ENVIAR,300100,20/02/2023 12:02,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250
2,85,5,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,NO OPERATIVO,489,,mimet vv-70,02/02/2023 6:14,None,POR_ENVIAR,1002,02/02/2023 6:14,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250
3,82,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,NO OPERATIVO,486,,mimet vv-70,02/02/2023 6:14,None,POR_ENVIAR,150250,02/02/2023 6:14,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250


In [2488]:
equipos_envio = equipos.drop(['temperaturaMinima', 'OTRO', 'ESTADO', 'Observaciones','ESTADO_CORREO','temperaturaMaxima','Creado', 'DESCRIPCION', 'Modificado','created_at', 'updated_at', 'id' ], axis=1)
#display(equipos_envio)

### Cambiar nombre de columnas

In [2489]:
equipos_envio = equipos_envio.rename(
    columns={
        'nombreMaquina' : 'NUMERO MAQUINA',
        'nombreLocal'   : 'NOMBRE LOCAL',
        'NUMERO_SERIE'  : 'NUMERO SERIE',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
        'ORDEN_COMPRA'  : 'ORDEN TRABAJO'
    })
#display(equipos_envio)


### Cambiar nombre de local

In [2490]:
#equipos_envio['NOMBRE LOCAL'] = equipos_envio['NOMBRE LOCAL'].apply(lambda x: x.split('-')[1])
#display(equipos_envio)

### Cambiar datos de Local, tipo de equipo, de columnas

In [2491]:
unique_values = equipos_envio['NOMBRE LOCAL'].unique()
unique_values


array(['CHJ5118800 - DARKSTORE COSTANERA',
       'CHJ5028800 - DARKSTORE KENNEDY'], dtype=object)

In [2492]:
for i in unique_values:
    print(i)

CHJ5118800 - DARKSTORE COSTANERA
CHJ5028800 - DARKSTORE KENNEDY


# Obtención de data historica
# *Obtención de data historica*
# **Obtención de data historica**

In [2493]:
equiposHistorico = dbAzure.getQuerySolpedsHistorico()
#display(equiposHistorico)
print(equiposHistorico.columns)


/Users/camilo/Desktop/PROYECTOS/ROTACION/classes/dbAzure.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(self.query, self.connection)


Index(['id', 'Local', 'nombreMaquina', 'NUMERO_SERIE', 'Temperatura',
       'Creado_por', 'HORA', 'FechaReal', 'FECHA', 'nombreProducto',
       'temperaturaProducto', 'AC1', 'encargadoAC', 'TIPO_EQUIPO', 'Creado',
       'ZONA', 'EQUIPO', 'ESTADO', 'ANO', 'MES', 'DIA', 'CAMBIO_ESTADO',
       'nombreProducto2', 'temperaturaProducto2', 'CANTIDAD_FALLOS',
       'created_at', 'updated_at'],
      dtype='object')


In [2494]:
equiposHistorico = equiposHistorico.rename(
    columns={
        'nombreMaquina' : 'NUMERO MAQUINA',
        'Local'   : 'NOMBRE LOCAL',
        'NUMERO_SERIE'  : 'NUMERO SERIE',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
        'ORDEN_COMPRA'  : 'ORDEN TRABAJO',
        'Observaciones' : 'TIPO EQUIPO',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
    })
#display(equiposHistorico)

In [2495]:
equiposHistorico = equiposHistorico.drop(['CANTIDAD_FALLOS', 'NUMERO SERIE'], axis=1)


In [2496]:
print(equiposHistorico['ESTADO'].unique())

['*' 'PENDIENTE DE REVISIÓN' 'NO OPERATIVO' 'OPERATIVO' 'None']


In [2497]:
equipos_envio.dtypes

NUMERO MAQUINA     int64
NOMBRE LOCAL      object
TIPO EQUIPO       object
ZONA              object
NUMERO SERIE      object
OBSERVACION       object
MODELO            object
ORDEN TRABAJO     object
dtype: object

In [2498]:
equiposHistorico.dtypes

id                               int64
NOMBRE LOCAL                    object
NUMERO MAQUINA                  object
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
ZONA                            object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO                   object
nombreProducto2                 object
temperaturaProducto2            object
created_at              datetime64[ns]
updated_at              datetime64[ns]
dtype: object

# Quitar None

In [2499]:
equiposHistorico = equiposHistorico.dropna(subset=['NUMERO MAQUINA'])

In [2500]:
df_filtrado = equiposHistorico.loc[(equiposHistorico['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') & (equiposHistorico['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

# Tranforma a Int64 la columna 'NUMERO MAQUINA'

In [2501]:
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].replace(regex=r'[^0-9\-]+', value=-1)
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].fillna(-1)
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].astype('int64')

tipos_de_datos = equiposHistorico.dtypes
print(tipos_de_datos)

id                               int64
NOMBRE LOCAL                    object
NUMERO MAQUINA                   int64
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
ZONA                            object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO                   object
nombreProducto2                 object
temperaturaProducto2            object
created_at              datetime64[ns]
updated_at              datetime64[ns]
dtype: object


# No operativos en los 2 locales

In [2502]:
df_filtrado = equiposHistorico.loc[((equiposHistorico['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') | (equiposHistorico['NOMBRE LOCAL'] == 'CHJ5118800 - DARKSTORE COSTANERA')) & (equiposHistorico['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)


# Unir las tablas 

In [2503]:
base_equipos = equipos_envio.join(equiposHistorico.set_index(['NOMBRE LOCAL', 'NUMERO MAQUINA', 'ZONA']), on=['NOMBRE LOCAL', 'NUMERO MAQUINA', 'ZONA'], how='left')
#display(base_equipos)

In [2504]:
df_filtrado = base_equipos.loc[((base_equipos['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') | (base_equipos['NOMBRE LOCAL'] == 'CHJ5118800 - DARKSTORE COSTANERA')) & (base_equipos['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

In [2505]:
print(base_equipos.value_counts('ESTADO'))


ESTADO
*                        55
None                     14
NO OPERATIVO             11
PENDIENTE DE REVISIÓN     6
OPERATIVO                 2
dtype: int64


In [2506]:
df_filtrado = base_equipos.loc[(base_equipos['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') & (base_equipos['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

In [2507]:
#display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

In [2508]:
tipos_de_datos = base_equipos.dtypes
print(tipos_de_datos)

NUMERO MAQUINA                   int64
NOMBRE LOCAL                    object
TIPO EQUIPO                     object
ZONA                            object
NUMERO SERIE                    object
OBSERVACION                     object
MODELO                          object
ORDEN TRABAJO                   object
id                               int64
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO            

# Eliminar columnas

In [2509]:
base_equipos = base_equipos.drop([
        'id',
        'Temperatura',
        'Creado_por',
        'Creado',
        'nombreProducto','temperaturaProducto', 'AC1',
        'Creado',
        'temperaturaProducto2',
        'created_at',
        'encargadoAC',
        'updated_at',
        'CAMBIO_ESTADO',
        'TIPO_EQUIPO',
        'nombreProducto2',
        'FechaReal',
        'HORA',
        'EQUIPO',
        'FECHA',
        'MODELO'
    ], axis=1)
#display(base_equipos)

In [2510]:
#display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

# Se ordenan los datos por local, zona y número de máquina

In [2511]:
base_equipos.sort_values(by=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'], ascending=[True, True, True], inplace=True)
#pd.set_option('display.max_rows', None)
#display(base_equipos)

In [2512]:
print(base_equipos.value_counts('ESTADO'))


ESTADO
*                        55
None                     14
NO OPERATIVO             11
PENDIENTE DE REVISIÓN     6
OPERATIVO                 2
dtype: int64


In [2513]:
df_filtrado = base_equipos.loc[base_equipos['ESTADO'] == 'NO OPERATIVO']
#display(df_filtrado)

In [2514]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,ANO,MES,DIA
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023,5,20
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023,4,11
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023,3,22
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,27
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,26
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,25
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,24
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,21
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,20
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,19


# Crear columna fecha

In [2515]:
base_equipos['FECHA'] = pd.to_datetime(base_equipos[['ANO', 'MES', 'DIA']].apply(lambda x: '-'.join(x.astype(str)), axis=1))
base_equipos = base_equipos.drop(['ANO', 'MES','DIA'], axis=1)
#display(base_equipos)


# Se añade un peso a cada registro (No operativo: 1, Otro:0)

In [2516]:
base_equipos['CUENTA_ESTADO'] = base_equipos['ESTADO'].apply(lambda x: 1 if x == 'NO OPERATIVO' else 0)
base_equipos['CUENTA_ESTADO_PENDIENTE'] = base_equipos['ESTADO'].apply(lambda x: 1 if x == 'PENDIENTE DE REVISIÓN' else 0)


# Ordenizar por fechas y promediar ponderador de No Operativo

In [2517]:
#cantidades = base_equipos.groupby('FECHA')['CUENTA_ESTADO'].transform('mean')
base_equipos['PONDERADOR NO OP'] = base_equipos.groupby(['FECHA', 'NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'])['CUENTA_ESTADO'].transform('mean')
base_equipos['PONDERADOR PENDIENTE'] = base_equipos.groupby(['FECHA', 'NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'])['CUENTA_ESTADO_PENDIENTE'].transform('mean')

display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0,0.00,0.0


In [2518]:
base_equipos = base_equipos.sort_values(by='FECHA')
display(base_equipos)


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2020-01-17,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0,0.00,0.0


# Eliminar registros sobrantes del día (solo se deja un registro por día)

In [2519]:
base_equipos.drop_duplicates(subset=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA', 'NUMERO SERIE','FECHA'], inplace=True)
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2020-01-17,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0,0.00,0.0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-07,0,0,0.00,0.0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-10,0,0,0.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0,0.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.00,1.0


In [2520]:
display(base_equipos[base_equipos['ZONA'] == 'Frutas y Verduras refrigeradas'])


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2020-01-17,0,0,0.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0,0.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0,0.25,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2023-03-22,0,0,0.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,1,0.00,1.0


### Tipos de 'ESTADO'

In [2521]:
print(base_equipos['ESTADO'].unique())

['OPERATIVO' 'None' '*' 'PENDIENTE DE REVISIÓN' 'NO OPERATIVO']


In [2522]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0,0.25,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-24,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-25,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-26,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-27,1,0,1.00,0.0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,NO OPERATIVO,2023-04-04,1,0,0.50,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023-04-11,1,0,1.00,0.0


# Equipos de envío base

In [2523]:
equipos_envio.sort_values(by=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'], ascending=[True, True, True], inplace=True)
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731



# Se añade las columnas para el reporte

In [2524]:
equipos_envio['DIAS MALOS HISTORICO'] = 0
equipos_envio['CANTIDAD FALLAS'] = 0
equipos_envio['DIAS ULTIMA FALLA'] = 0
#display(equipos_envio)

# Se obtiene la cantidad de registros no operativos

In [2525]:
cantidades_noOP = base_equipos.loc[base_equipos['PONDERADOR NO OP'] > 0].groupby(['NUMERO MAQUINA', 'NOMBRE LOCAL', 'ZONA'])['ESTADO'].count()
print(cantidades_noOP)

NUMERO MAQUINA  NOMBRE LOCAL                      ZONA                          
2               CHJ5028800 - DARKSTORE KENNEDY    Fiambres y Quesos                 2
                                                  Frutas y Verduras refrigeradas    1
5               CHJ5028800 - DARKSTORE KENNEDY    Frutas y Verduras refrigeradas    7
8               CHJ5118800 - DARKSTORE COSTANERA  Congelados                        1
Name: ESTADO, dtype: int64


# Añadir cantidad de días no operativos a base

In [2526]:
equipos_envio['DIAS MALOS HISTORICO'] = cantidades_noOP.values
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO,DIAS MALOS HISTORICO,CANTIDAD FALLAS,DIAS ULTIMA FALLA
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100,2,0,0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250,1,0,0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002,7,0,0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731,1,0,0


# **FALTAN PRUEBAS DE LOS DÍAS NO OPERATIVOS**

# **OBTENCIÓN DE CANTIDAD DE FALLAS POR EQUIPO**

# Contabilizar los PENDIENTE DE APROBACIÓN a NO OPERATIVO

In [2527]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0,0.25,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-24,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-25,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-26,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-27,1,0,1.00,0.0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,NO OPERATIVO,2023-04-04,1,0,0.50,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023-04-11,1,0,1.00,0.0


In [2528]:
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2020-01-17,0,0,0.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0,0.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0,0.00,0.0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-07,0,0,0.00,0.0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-10,0,0,0.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.00,0.0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0,0.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.00,1.0


In [2529]:
display(base_equipos[(base_equipos['ESTADO'] == 'NO OPERATIVO') | (base_equipos['ESTADO'] == 'PENDIENTE DE REVISIÓN')])


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0,0.25,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-24,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-25,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-26,1,0,1.00,0.0


In [2530]:
display(base_equipos[(base_equipos['PONDERADOR NO OP'] > 0) | (base_equipos['PONDERADOR PENDIENTE'] > 0)])


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0,0.25,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,1,0.00,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-24,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-25,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-26,1,0,1.00,0.0


# Cantidad de veces que ocurre el cambio de PENDIENTE DE REVISIÓN a NO OPERATIVO

In [2531]:
# Crear un dataframe con el estado al día siguiente

base_equipos_next_day = base_equipos.copy()
base_equipos_next_day['FECHA'] -= pd.Timedelta(days=1)

# Unir el dataframe original con el dataframe que tiene el estado al día siguiente
df_merged = base_equipos.merge(base_equipos_next_day, on=['NUMERO MAQUINA', 'NOMBRE LOCAL', 'ZONA', 'FECHA'], suffixes=('', '_siguiente'))
display(df_merged)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,...,PONDERADOR PENDIENTE,TIPO EQUIPO_siguiente,NUMERO SERIE_siguiente,OBSERVACION_siguiente,ORDEN TRABAJO_siguiente,ESTADO_siguiente,CUENTA_ESTADO_siguiente,CUENTA_ESTADO_PENDIENTE_siguiente,PONDERADOR NO OP_siguiente,PONDERADOR PENDIENTE_siguiente
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,...,1.0,Equipo Refrigerado,1745058,,1002,NO OPERATIVO,1,0,1.00,0.0
1,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,...,0.0,Equipo Refrigerado,1745058,,1002,NO OPERATIVO,1,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,...,0.0,Equipo Refrigerado,1745058,,1002,NO OPERATIVO,1,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,...,0.0,Equipo Refrigerado,sin n°de serie,,300100,*,0,0,0.00,0.0
4,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,...,1.0,Equipo Refrigerado,1745057,,150250,NO OPERATIVO,1,0,0.25,0.0
5,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,...,0.0,Equipo Refrigerado,1745058,,1002,OPERATIVO,0,0,0.00,0.0
6,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2023-03-22,0,...,0.0,Equipo Refrigerado,1745058,,1002,PENDIENTE DE REVISIÓN,0,1,0.00,1.0
7,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,...,0.0,Equipo Refrigerado,sin n°de serie,,300100,*,0,0,0.00,0.0
8,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,...,1.0,Equipo Refrigerado,1745058,,1002,NO OPERATIVO,1,0,1.00,0.0
9,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,...,0.0,Equipo Refrigerado,sin n°de serie,,300100,*,0,0,0.00,0.0


In [2532]:
df_merged.columns

Index(['NUMERO MAQUINA', 'NOMBRE LOCAL', 'TIPO EQUIPO', 'ZONA', 'NUMERO SERIE',
       'OBSERVACION', 'ORDEN TRABAJO', 'ESTADO', 'FECHA', 'CUENTA_ESTADO',
       'CUENTA_ESTADO_PENDIENTE', 'PONDERADOR NO OP', 'PONDERADOR PENDIENTE',
       'TIPO EQUIPO_siguiente', 'NUMERO SERIE_siguiente',
       'OBSERVACION_siguiente', 'ORDEN TRABAJO_siguiente', 'ESTADO_siguiente',
       'CUENTA_ESTADO_siguiente', 'CUENTA_ESTADO_PENDIENTE_siguiente',
       'PONDERADOR NO OP_siguiente', 'PONDERADOR PENDIENTE_siguiente'],
      dtype='object')

In [2533]:
df_merged = df_merged.drop([
    'TIPO EQUIPO_siguiente',
    'NUMERO SERIE_siguiente',
    'OBSERVACION_siguiente',
    'ORDEN TRABAJO_siguiente',
    'CUENTA_ESTADO_siguiente',
    
    ], axis=1)

display(df_merged)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE,ESTADO_siguiente,CUENTA_ESTADO_PENDIENTE_siguiente,PONDERADOR NO OP_siguiente,PONDERADOR PENDIENTE_siguiente
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0
1,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.0,0.0,NO OPERATIVO,0,1.00,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.0,0.0,NO OPERATIVO,0,1.00,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0,0.0,0.0,*,0,0.00,0.0
4,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.0,1.0,NO OPERATIVO,0,0.25,0.0
5,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,0,1.0,0.0,OPERATIVO,0,0.00,0.0
6,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2023-03-22,0,0,0.0,0.0,PENDIENTE DE REVISIÓN,1,0.00,1.0
7,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0,0.0,0.0,*,0,0.00,0.0
8,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0
9,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0,0.0,0.0,*,0,0.00,0.0


# Se obtienen los registros donde un día es 'PENDIENTE DE REVISIÓN y al otro es 'NO OPERATIVO'

In [2534]:
# Agregar una nueva columna 'VECES_CAMBIO' en función de la condición deseada.
df_merged['VECES_CAMBIO'] = (df_merged['PONDERADOR PENDIENTE'] > 0) & (df_merged['PONDERADOR NO OP_siguiente'] > 0)

# Convertir la columna 'VECES_CAMBIO' a enteros (True se convierte en 1, False se convierte en 0).
df_merged['VECES_CAMBIO'] = df_merged['VECES_CAMBIO'].astype(int)

display(df_merged)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE,ESTADO_siguiente,CUENTA_ESTADO_PENDIENTE_siguiente,PONDERADOR NO OP_siguiente,PONDERADOR PENDIENTE_siguiente,VECES_CAMBIO
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1
1,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,0,1.0,0.0,NO OPERATIVO,0,1.00,0.0,0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,0,1.0,0.0,NO OPERATIVO,0,1.00,0.0,0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0,0.0,0.0,*,0,0.00,0.0,0
4,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.0,1.0,NO OPERATIVO,0,0.25,0.0,1
5,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,0,1.0,0.0,OPERATIVO,0,0.00,0.0,0
6,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2023-03-22,0,0,0.0,0.0,PENDIENTE DE REVISIÓN,1,0.00,1.0,0
7,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0,0.0,0.0,*,0,0.00,0.0,0
8,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1
9,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0,0.0,0.0,*,0,0.00,0.0,0


## Cantidad de veces que cambia de PENDIENTE DE REVISIÓN a NO OPERATIVO

In [2535]:
df_grouped = df_merged.groupby(['NOMBRE LOCAL','NUMERO MAQUINA','ZONA'])['VECES_CAMBIO'].sum().reset_index()
display(df_grouped)

,NOMBRE LOCAL,NUMERO MAQUINA,ZONA,VECES_CAMBIO
0,CHJ5028800 - DARKSTORE KENNEDY,2,Fiambres y Quesos,1
1,CHJ5028800 - DARKSTORE KENNEDY,2,Frutas y Verduras refrigeradas,1
2,CHJ5028800 - DARKSTORE KENNEDY,5,Frutas y Verduras refrigeradas,2
3,CHJ5118800 - DARKSTORE COSTANERA,8,Congelados,1


## Se añade esta información a CANTIDAD FALLAS

In [2536]:
equipos_envio = equipos_envio.merge(df_grouped, on=['NUMERO MAQUINA', 'NOMBRE LOCAL', 'ZONA'])
equipos_envio['CANTIDAD FALLAS'] = equipos_envio['VECES_CAMBIO']
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO,DIAS MALOS HISTORICO,CANTIDAD FALLAS,DIAS ULTIMA FALLA,VECES_CAMBIO
0,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100,2,1,0,1
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250,1,1,0,1
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002,7,2,0,2
3,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731,1,1,0,1


# **OBTENER DÍAS DE FALLAS DE LA ÚLTIMA VEZ QUE ESTUVO MALO** 

## Ver si puedo obtener la fecha que utilicé (parece que tengo que pasar a fecha + 1 para obtener esa fecha)

## Obtener los registros en los que hubo un cambio de NO OPERATIVO a PENDIENTE DE REVISIÓN

In [2537]:
df_fechas_de_cambio = df_merged[df_merged['VECES_CAMBIO'] > 0]
display(df_fechas_de_cambio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE,ESTADO_siguiente,CUENTA_ESTADO_PENDIENTE_siguiente,PONDERADOR NO OP_siguiente,PONDERADOR PENDIENTE_siguiente,VECES_CAMBIO
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1
4,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.0,1.0,NO OPERATIVO,0,0.25,0.0,1
8,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1
18,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,PENDIENTE DE REVISIÓN,2023-04-03,0,1,0.0,1.0,NO OPERATIVO,0,0.50,0.0,1
24,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,PENDIENTE DE REVISIÓN,2023-04-10,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1


# Se ordenan las fechas de la más cercana a más lejana, y solo se mantiene la más cercana (fecha de última falla)

In [2538]:
# Ordena el DataFrame por FECHA en orden descendente
df = df_fechas_de_cambio.sort_values(by='FECHA', ascending=False)
display(df)
# Elimina los registros duplicados basándote en las columnas NOMBRE LOCAL, NUMERO MAQUINA y ZONA, y conserva el primer registro (el de fecha más reciente)
df = df_fechas_de_cambio.drop_duplicates(subset=['NOMBRE LOCAL', 'NUMERO MAQUINA', 'ZONA'], keep='first')

display(df)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE,ESTADO_siguiente,CUENTA_ESTADO_PENDIENTE_siguiente,PONDERADOR NO OP_siguiente,PONDERADOR PENDIENTE_siguiente,VECES_CAMBIO
24,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,PENDIENTE DE REVISIÓN,2023-04-10,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1
18,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,PENDIENTE DE REVISIÓN,2023-04-03,0,1,0.0,1.0,NO OPERATIVO,0,0.50,0.0,1
8,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1
4,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.0,1.0,NO OPERATIVO,0,0.25,0.0,1
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE,ESTADO_siguiente,CUENTA_ESTADO_PENDIENTE_siguiente,PONDERADOR NO OP_siguiente,PONDERADOR PENDIENTE_siguiente,VECES_CAMBIO
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1
4,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.0,1.0,NO OPERATIVO,0,0.25,0.0,1
18,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,PENDIENTE DE REVISIÓN,2023-04-03,0,1,0.0,1.0,NO OPERATIVO,0,0.50,0.0,1
24,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,PENDIENTE DE REVISIÓN,2023-04-10,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1


## Obtener días desde la de PENDIENTE DE REVISIÓN (justo antes del día no operativo) hasta hoy

In [2539]:
# Obtiene la fecha actual
hoy = pd.Timestamp.now()

# Calcula la diferencia en días entre la columna FECHA y el día de hoy
df['DIFERENCIA_DIAS'] = (hoy - df['FECHA']).dt.days - 1

display(df)

/var/folders/rs/5wl8vzfj4vz7xxdjjr9hk3800000gn/T/ipykernel_2809/1832111179.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCIA_DIAS'] = (hoy - df['FECHA']).dt.days - 1


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,CUENTA_ESTADO_PENDIENTE,PONDERADOR NO OP,PONDERADOR PENDIENTE,ESTADO_siguiente,CUENTA_ESTADO_PENDIENTE_siguiente,PONDERADOR NO OP_siguiente,PONDERADOR PENDIENTE_siguiente,VECES_CAMBIO,DIFERENCIA_DIAS
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1,28
4,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,1,0.0,1.0,NO OPERATIVO,0,0.25,0.0,1,25
18,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,PENDIENTE DE REVISIÓN,2023-04-03,0,1,0.0,1.0,NO OPERATIVO,0,0.50,0.0,1,12
24,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,PENDIENTE DE REVISIÓN,2023-04-10,0,1,0.0,1.0,NO OPERATIVO,0,1.00,0.0,1,5


# Añadir cantidad de días de último fallo a equipos_envio

In [2540]:
df = df[['NUMERO MAQUINA','NOMBRE LOCAL', 'ZONA', 'DIFERENCIA_DIAS']]
equipos_envio = equipos_envio.merge(df, on=['NUMERO MAQUINA', 'NOMBRE LOCAL', 'ZONA'])
equipos_envio['DIAS ULTIMA FALLA'] = equipos_envio['DIFERENCIA_DIAS']

equipos_envio = equipos_envio.drop(columns=['VECES_CAMBIO','DIFERENCIA_DIAS'])
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO,DIAS MALOS HISTORICO,CANTIDAD FALLAS,DIAS ULTIMA FALLA
0,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100,2,1,5
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250,1,1,25
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002,7,2,28
3,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731,1,1,12


# **---------------------------------------------------------------------------**

In [2541]:
rutaRelativa.replace('/PROYECTOS','/PROYECTOS/ROTACION/DOCS')

'/Users/camilo/Desktop/PROYECTOS/ROTACION/DOCS'

In [2542]:
wb = Workbook()
ws = wb.active
ws.title = "EQUIPOS_NO_OPERATIVOS"

# Convierte el DataFrame en filas y agrega las filas a la hoja de trabajo
for r in dataframe_to_rows(equipos_envio, index=False, header=True):
    ws.append(r)

# Ajusta el ancho de las columnas según el contenido
for col in ws.columns:
    max_length = 0
    column = get_column_letter(col[0].column)  # Obtiene la letra de la columna
    for cell in col:
        max_length = max(max_length, len(str(cell.value)))
    adjusted_width = (max_length + 2) * 1.2  # Ajusta el ancho de la columna
    ws.column_dimensions[column].width = adjusted_width

# Guarda el archivo de Excel
wb.save("EQUIPOS_NO_OPERATIVOS.xlsx")

In [2543]:
equipos_envio.to_excel("EQUIPOS_NO_OPERATIVOS.xlsx", auto_fit_columns=True)
#remove("EQUIPOS_NO_OPERATIVOS.xlsx")


TypeError: NDFrame.to_excel() got an unexpected keyword argument 'auto_fit_columns'

In [2544]:
rutaRelativa = rutaRelativa.replace('/PROYECTOS', '/PROYECTOS/ROTACION')
rutaRelativa


'/Users/camilo/Desktop/PROYECTOS/ROTACION'

In [2545]:
correos         = correosDestino.correosDestino(estadoProduccion)
correos['to'].append('camilomansillaulloa@gmail.com')
correos

{'to': ['camilomansillaulloa@gmail.com'],
 'cc': [],
 'bcc': ['camilo.mansilla@cencosud.cl']}

In [2546]:
archivos    = []
archivos.append({
    'nombre_archivo':   'EQUIPOS_NO_OPERATIVOS.xlsx',
    'tipo_archivo':     'application',
    'formato_archivo': 'vnd.ms-excel',
    'ruta':             f'{rutaRelativa}/EQUIPOS_NO_OPERATIVOS.xlsx'
})
correoEnvioArchivo('EQUIPOS NO OPERATIVOS DS', "", correos, archivos)

In [ ]:
equipos['ESTADO_CORREO'] = 'NO ENVIAR'
display(equipos)

,id,nombreMaquina,temperaturaMinima,temperaturaMaxima,nombreLocal,TIPO_DE_EQUIPO,ZONA,NUMERO_SERIE,ESTADO,OTRO,OBSERVACION,MODELO,Creado,DESCRIPCION,ESTADO_CORREO,ORDEN_COMPRA,Modificado,Observaciones,created_at,updated_at
0,338,8,-18,-12,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,NO OPERATIVO,0,prueba,,,,NO ENVIAR,3778731,,,2023-03-22 11:23:40.000,2023-03-22 11:23:40.000
1,90,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,NO OPERATIVO,494,,None,02/02/2023 6:14,None,NO ENVIAR,300100,20/02/2023 12:02,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250
2,85,5,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,NO OPERATIVO,489,,mimet vv-70,02/02/2023 6:14,None,NO ENVIAR,1002,02/02/2023 6:14,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250
3,82,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,NO OPERATIVO,486,,mimet vv-70,02/02/2023 6:14,None,NO ENVIAR,150250,02/02/2023 6:14,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250


In [ ]:
for index in equipos.index:
    dbAzure.changeStatusSolped(equipos.iloc[index])

1 record(s) affected
1 record(s) affected
1 record(s) affected
1 record(s) affected


In [ ]:
#fileHtml    = open(f'{rutaRelativa}/templates/CorreoGerenteDS.html', 'r', encoding='utf-8')
#htmlBase    = fileHtml.read()
#fileHtml.close()
#htmlBase

In [ ]:
#fileHtml    = open(f'{rutaRelativa}/templates/correoAprobacionGerenteBanderas.html', 'r', encoding='utf-8')
#htmlBase    = fileHtml.read()
#fileHtml.close()

In [ ]:
#solpeds = dbAzure.getQuerySolpeds()
#solpeds